# Train rt-detr to detect weeds in potato fields

In this notebook we'll train a [rt-detr](https://docs.ultralytics.com/models/rtdetr/) model to detect weeds in a potato fields. 
The target weeds are:
- kikuyo
- lengua de vaca
- diente de león

These weeds are common in potatoes fields. The images was captured in 9-10 meters height with a drone. Then, the images was selected and cropped 
to 1920x1920 pixels. Finally, the images was labeled with Roboflow.

## Install dependencies
First of all, we'll check the GPU, and install the dependencies.

In [ ]:
!python --version
!cat /etc/os-release

In [ ]:
!pip install -q coco-eval pycocotools supervision ultralytics
!pip install -q wandb ipywidgets --upgrade

## Initialize weights & biases
We'll use weights & biases to log the training process.

In [ ]:
import wandb
from kaggle_secrets import UserSecretsClient

project_name = 'rt-detr-potatoes-final'
name = 'train-default-config'
user_secrets = UserSecretsClient()
key = user_secrets.get_secret('wandb')


wandb.login(key=key)
wandb.init(project=project_name, name=name, job_type='training')

## Create data.yaml
We'll create a data.yaml file with the classes and the paths to the images.

In [ ]:
import yaml

data_yaml = '''
path: /kaggle/input/potatoes-weeds-dataset/potatoes-dataset-final.v2.yolov8

train: train/images
val: valid/images
test: test/images

names:
  0: diente_leon
  1: kikuyo
  2: lengua_vaca
  3: otros
  4: papa


# Train settings -------------------------------------------------------------------------------------------------------
imgsz: 640 # (int | list) input images size as int for train and val modes, or list[w,h] for predict and export modes
epochs: 100 # (int) number of epochs to train for
patience: 100 # (int) epochs to wait for no observable improvement for early stopping of training
batch: 8 # (int) number of images per batch (-1 for AutoBatch)
optimizer: auto # (str) optimizer to use, choices=[SGD, Adam, Adamax, AdamW, NAdam, RAdam, RMSProp, auto]

# Val/Test settings ----------------------------------------------------------------------------------------------------
split: val # (str) dataset split to use for validation, i.e. 'val', 'test' or 'train'
save_json: True # (bool) save results to JSON file
iou: 0.7 # (float) intersection over union (IoU) threshold for NMS
max_det: 300 # (int) maximum number of detections per image

# Hyperparameters ------------------------------------------------------------------------------------------------------
lr0: 0.01 # (float) initial learning rate (i.e. SGD=1E-2, Adam=1E-3)
lrf: 0.01 # (float) final learning rate (lr0 * lrf)
momentum: 0.937 # (float) SGD momentum/Adam beta1
weight_decay: 0.0005 # (float) optimizer weight decay 5e-4
hsv_h: 0.015 # (float) image HSV-Hue augmentation (fraction)
hsv_s: 0.7 # (float) image HSV-Saturation augmentation (fraction)
hsv_v: 0.4 # (float) image HSV-Value augmentation (fraction)
degrees: 0.0 # (float) image rotation (+/- deg)
translate: 0.1 # (float) image translation (+/- fraction)
flipud: 0.0 # (float) image flip up-down (probability)
fliplr: 0.5 # (float) image flip left-right (probability)
mosaic: 1.0 # (float) image mosaic (probability)
erasing: 0.4 # (float) random cutout (probability)
crop_fraction: 1.0 # (float) fraction of image to crop
'''

data = yaml.safe_load(data_yaml)

with open('data.yaml', 'w') as file:
    yaml.dump(data, file)

## Prepare model
We'll download the rt-detr model and prepare it before training.

In [ ]:
from ultralytics import RTDETR
from wandb.integration.ultralytics import add_wandb_callback


# Load a COCO-pretrained RT-DETR-l model
model = RTDETR('rtdetr-l.pt')


# Add W&B Callback for Ultralytics
add_wandb_callback(model, enable_model_checkpointing=True)

## Train the model
We'll train the model.

In [ ]:
results = model.train(
    project=project_name, name=name, data='data.yaml', device=[0, 1], batch=8, augment=True
)
results = model.val()

wandb.finish()

## Save the model
We'll save the model in the output folder.

In [ ]:
model.export()